# IMPORTANT 
Make sure you've imported the [environment.yml](https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python/blob/main/notebooks/environment.yml) file provided in the git repo on the left sidebar.

This will ensure if you have the right packages needed to run this Notebook.

## 1. Data Ingestion

The `diamonds` dataset has been widely used in data science and machine learning. We will use it to demonstrate Snowflake's native data science transformers in terms of database functionality and Spark & Pandas comportablity, using non-synthetic and statistically appropriate data that is well known to the ML community.



### Import Libraries

In [ ]:
# Snowpark for Python
from snowflake.snowpark.types import DoubleType
import snowflake.snowpark.functions as F

### Setup and establish Secure Connection to Snowflake

Notebooks establish a Snowpark Session when the notebook is attached to the kernel. We create a new warehouse, database, and schema that will be used throughout this tutorial.

In [ ]:
-- Using Warehouse, Database, and Schema created during Setup
USE WAREHOUSE ML_HOL_WH;
USE DATABASE ML_HOL_DB;
USE SCHEMA ML_HOL_SCHEMA;

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"e2e_ml_snowparkpython", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))

### Use the Snowpark DataFrame Reader to read in data from the externally staged `diamonds` CSV file 

In setup.sql, we staged the `diamonds.csv` file from an external s3 bucket. Now, we can read it in.

For more information on loading data, see documentation on [snowflake.snowpark.DataFrameReader](https://docs.snowflake.com/ko/developer-guide/snowpark/reference/python/api/snowflake.snowpark.DataFrameReader.html).




In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
diamonds_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@DIAMONDS_ASSETS")

diamonds_df

In [ ]:
# Look at descriptive stats on the DataFrame
diamonds_df.describe()

In [ ]:
diamonds_df.columns

### Data cleaning

First, let's force headers to uppercase using Snowpark DataFrame operations for standardization when columns are later written to a Snowflake table.

In [ ]:
# Force headers to uppercase
for colname in diamonds_df.columns:
    if colname == '"table"':
       new_colname = "TABLE_PCT"
    else:
        new_colname = str.upper(colname)
    diamonds_df = diamonds_df.with_column_renamed(colname, new_colname)

diamonds_df

Next, we standardize the category formatting for `CUT` using Snowpark DataFrame operations.

This way, when we write to a Snowflake table, there will be no inconsistencies in how the Snowpark DataFrame will read in the category names. Secondly, the feature transformations on categoricals will be easier to encode.

In [ ]:
def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))

for col in ["CUT"]:
    diamonds_df = diamonds_df.with_column(col, fix_values(col))

diamonds_df

Check the schema.

In [ ]:
list(diamonds_df.schema)

Finally, let's cast the decimal types to DoubleType() since DecimalType() isn't support by Snowpark ML at the moment.

In [ ]:
for colname in ["CARAT", "X", "Y", "Z", "DEPTH", "TABLE_PCT"]:
    diamonds_df = diamonds_df.with_column(colname, diamonds_df[colname].cast(DoubleType()))

diamonds_df

### Write cleaned data to a Snowflake table

In [ ]:
diamonds_df.write.mode('overwrite').save_as_table('diamonds')

In the next notebook, we will perform data transformations with the Snowpark ML Preprocessing API for feature engineering. 